# INFO 4300: P04, Prototype 2

In [ ]:
# importing necessary packages for analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import duckdb
from wordcloud import WordCloud
import re

In [ ]:
# Importing dataset
cols = ['url','Perfume','Brand','Country','Gender','Rating Value', \
        'Rating Count','Year','Top','Middle','Base',
        'mainaccord1','mainaccord2','mainaccord3','mainaccord4','mainaccord5']

frag_kaggle = pd.read_csv('fra_cleaned.csv',
    usecols = cols,  encoding='latin-1', delimiter=';')
print(len(frag_kaggle))
frag_kaggle.head()

24063


,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,https://www.fragrantica.com/perfume/xerjoff/ac...,accento-overdose-pride-edition,xerjoff,Italy,unisex,"1,42",201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",rose,woody,fruity,aromatic,floral
1,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2024,jean-paul-gaultier,France,women,"1,86",70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",citrus,white floral,sweet,fresh,musky
2,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2023,jean-paul-gaultier,France,unisex,"1,91",285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",citrus,white floral,sweet,fresh spicy,musky
3,https://www.fragrantica.com/perfume/bruno-bana...,pride-edition-man,bruno-banani,Germany,men,"1,92",59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",fruity,nutty,woody,tropical,NaN
4,https://www.fragrantica.com/perfume/jean-paul-...,le-male-pride-collector,jean-paul-gaultier,France,men,"1,93",632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",aromatic,warm spicy,fresh spicy,cinnamon,vanilla


In [ ]:
# data cleaning
frag_kaggle['Perfume'] = frag_kaggle['Perfume'].str.replace('-', ' ')
frag_kaggle['Brand'] = frag_kaggle['Brand'].str.replace('-', ' ')
frag_kaggle['Rating Value'] = frag_kaggle['Rating Value'].str.replace(',', '.')
frag_kaggle['Year'] = frag_kaggle['Year'].fillna(0).astype(int)

# removing " notes" & stripping trailing spaces
for col in ['Top', 'Middle', 'Base']:
    frag_kaggle[col] = frag_kaggle[col].fillna("").str.replace(" notes", "", case=False).str.strip()

# merging notes into one col
def notes_intersect(row):
    top_notes = set(row["Top"].split(", "))
    middle_notes = set(row["Middle"].split(", "))
    base_notes = set(row["Base"].split(", "))
    common_notes = top_notes | middle_notes | base_notes
    return ", ".join(note.strip() for note in common_notes if note) if common_notes else None
frag_kaggle["Notes"] = frag_kaggle.apply(notes_intersect, axis=1)

# replacing null accord values
accord_cols = ['mainaccord1', 'mainaccord2', 'mainaccord3', 'mainaccord4', 'mainaccord5']
frag_kaggle[accord_cols] = frag_kaggle[accord_cols].fillna("")

# merging accords into one col
frag_kaggle["Accords"] = frag_kaggle[accord_cols].apply(
    lambda row: ", ".join(filter(None, [x.strip() for x in row if x.strip()])),
    axis=1
)

# dropping columns with missing values
pd.set_option('display.max_colwidth', None)
frag_clean2 = frag_kaggle.dropna(axis=1, how='all')
print(len(frag_clean2))
print(len(frag_kaggle))
frag_clean2.head(20)
frag_clean2.to_csv("frag_clean2.csv", index=False)

24063
24063
